In [1]:
# imports and constants
import json
import pickle
import datetime as dt

import gmaps
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import multiprocessing
import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns

from IPython.display import display, HTML
import ipywidgets as widgets

# user imports
from utils.process_data import *
from utils.stats import *
from utils.lifesense_utils import *

%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

# Location

## Circadean Rhythm Window Processing

In [18]:
def build_circ_window(pid, wk, start_date, end_date):
    """Builds circadean movement features within the given window.
    
    TODO refactor with build_circadean_stats
    
    Assumes that the start_date and end_date fall within the given week.
    
    
    Args:
        - pid (str): the participant id
        - wk (int): the study week to pull data from: 1, 4, 7, 10, 13, 16
        - fus_df (pd.DataFrame): a fus_df frame with properly formatted date columns
        - start_date (str): start date, yyyy-mm-dd
        - end_date (str): end date, yyyy-mm-dd
        
    Returns:
        - circ_df (pd.DataFrame): frame with circadean movement statistics calculated for the given time window
    """
    print(pid)
    loc = "/data/tliu/wk{}_ls_data/pdk-location/{}.df".format(wk, pid)
    fus_df = pd.read_pickle(loc)
    if fus_df.shape[0] < 1:
        return
    
    fus_df = format_raw_fus(fus_df)
    fus_df = fus_df[(fus_df['date'] >= start_date) & (fus_df['date'] <= end_date)]
    print(fus_df.shape)
    fus_df["is_wkday"] = (pd.to_datetime(fus_df['date']).dt.dayofweek < 5).astype(float)

    circ_df = pd.DataFrame()
    circ_df['pid'] = [pid]
    circ_df['circ_movt_tot'] = get_circadian_movement(fus_df) 
    circ_df['circ_movt_wkday'] = get_circadian_movement(fus_df.loc[fus_df['is_wkday'] == 1]) 
    circ_df['circ_movt_wkend'] = get_circadian_movement(fus_df.loc[fus_df['is_wkday'] == 0]) 
    
    return circ_df


def build_circ_dict(seq_df, target, pre_days=3, post_days=3):
    """Builds a (col, [vals]) dictionary for circ features.
    
    Has to be processed separately because circadean movement cannot be aggregated to daily values.
    """
    col_dict = {}

    cols = ['circ_movt_tot', 'circ_movt_wkday', 'circ_movt_wkend']
    for col in cols:
        col_dict[col] = []

    for idx, row in seq_df.iterrows():
        if pd.isna(row[target]):
            for col in cols:
                col_dict[col].append(np.nan)
            continue
        else:
            date = row['date']
            pid = row['pid']
            wk = row['study_wk']
            sel_df = build_circ_window(pid, wk, (date.floor('D') - pd.Timedelta(pre_days, unit='D')),                                      
                                                (date.floor('D') + pd.Timedelta(post_days, unit='D')))
            for col in cols:
                if sel_df is not None:
                    col_dict[col].append(sel_df[col])
                else:
                    col_dict[col].append(np.nan)
                
    return col_dict


In [19]:
%%time

test = build_circ_window('56596866', 4, '2019-08-15', '2019-09-01')
test

56596866
(3846, 5)
CPU times: user 488 ms, sys: 4.15 ms, total: 493 ms
Wall time: 484 ms


In [20]:
test.head()

,pid,circ_movt_tot,circ_movt_wkday,circ_movt_wkend
0,56596866,-10.657614,-9.976178,-14.412289


In [21]:
%%time

shift_df = pd.read_pickle("ls_data/phq_cluster_shift.df")
circ_dict = build_circ_dict(shift_df, 'phq_start', pre_days=14, post_days=-1)

00746649
(2355, 5)
00746649
(2043, 5)
00746649
(2284, 5)
00746649
(2497, 5)
00746649
(2731, 5)
00746649
(1394, 5)
01225297
(1505, 5)
01225297
(0, 5)
01225297


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/scipy/signal/spectral.py:151: RuntimeWarning: Mean of empty slice.
  pgram = _lombscargle(x, y - y.mean(), freqs)
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(1597, 5)
01225297
(904, 5)
01254121
(1313, 5)
01254121
(2436, 5)
01254121
(2731, 5)
01254121
(2881, 5)
01254121
(2894, 5)
01254121
(2941, 5)
01495950
(1263, 5)
01495950
(1275, 5)
01495950
(1199, 5)
01495950
(2735, 5)
01495950
(2698, 5)
01495950
(2666, 5)
01766910
(525, 5)
01766910
(1531, 5)
01766910
(2906, 5)
01766910
(2978, 5)
01766910
(2856, 5)
01766910
(2908, 5)
02970060
(2057, 5)
02970060
(2960, 5)
02970060
(3021, 5)
02970060
(2720, 5)
02970060
(2827, 5)
02970060
(2868, 5)
03233601
(1646, 5)
03233601
(2046, 5)
03233601
(3016, 5)
03233601
(2945, 5)
03233601
(3007, 5)
03578019
(1783, 5)
03578019
(2258, 5)
03578019
(2853, 5)
03578019
(2839, 5)
03578019
(3037, 5)
03939827
(295, 5)
03939827
(754, 5)
03939827
(327, 5)
03939827
(106, 5)
03939827
(891, 5)
04133537
(2181, 5)
04133537
(2965, 5)
04133537
(2910, 5)
04133537
(2808, 5)
04133537
(2941, 5)
04133537
(2035, 5)
04918121
(371, 5)
04918121
(429, 5)
04918121
(2649, 5)
04918121
(2750, 5)
04918121
(2922, 5)
04918121
(2991, 5)
06400675
(1

/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/scipy/signal/spectral.py:151: RuntimeWarning: Mean of empty slice.
  pgram = _lombscargle(x, y - y.mean(), freqs)
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



(484, 5)
11770862
(0, 5)
11927637
(1383, 5)


...

90638927
(2502, 5)
90638927
(2292, 5)
90638927
(2557, 5)
90638927
(2756, 5)
90638927
(1668, 5)
90763832
(2236, 5)
90763832
(3035, 5)
90763832
(3155, 5)
90763832
(3077, 5)
90763832
(3065, 5)
90763832
(3077, 5)
91048552
(2043, 5)
91048552
(2870, 5)
91048552
(2632, 5)
91048552
(1254, 5)
91048552
(1177, 5)
91048552


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/scipy/signal/spectral.py:151: RuntimeWarning: Mean of empty slice.
  pgram = _lombscargle(x, y - y.mean(), freqs)
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(1719, 5)
91788916
91788916
(0, 5)
93606382


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/scipy/signal/spectral.py:151: RuntimeWarning: Mean of empty slice.
  pgram = _lombscargle(x, y - y.mean(), freqs)
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(2133, 5)
93606382
(2251, 5)
93606382
(2886, 5)
93606382
(1607, 5)
93606382
(1553, 5)
93606382
(3026, 5)
93627939
(2075, 5)
93627939
(969, 5)
94277599
(2246, 5)
94277599
(2409, 5)
94277599
(2548, 5)
94277599
(2286, 5)
94277599
(1182, 5)
97678130
(1264, 5)
97678130
(1541, 5)
97678130
(2102, 5)
97678130
(1897, 5)
97678130
(355, 5)
97678130
(2353, 5)
98250113
(1132, 5)
98250113
(2331, 5)
98250113
(2729, 5)
98250113
(2718, 5)
98250113
(2704, 5)
98250113
(2742, 5)
98621494
(1603, 5)
98621494
(2848, 5)
98621494
(2775, 5)
98621494
(2850, 5)
98621494
(2374, 5)
99127649
(1396, 5)
99127649
(1626, 5)
99127649
(706, 5)
99127649
(3163, 5)
99127649
(1533, 5)
99338619
(1432, 5)
99338619
(2941, 5)
99338619
(3375, 5)
99338619
(3101, 5)
99338619
(1207, 5)
CPU times: user 5min 42s, sys: 21.3 s, total: 6min 3s
Wall time: 6min 5s


## Fused location correctness analysis

In [22]:
fus_daily = pd.read_pickle("ls_data/wk10/fus_daily.df")

In [25]:
fus_daily[fus_daily['velocity'] < 0]

,pid,date,dist,entropy,cluster,loc_var,velocity
16,61762096,2019-10-10,16801.665292,0.485634,29,7.57562,-29.441542


In [26]:
raw_fus = pd.read_pickle("/data/tliu/wk10_ls_data/pdk-location/61762096.df")

In [69]:
fus_df = format_raw_fus(raw_fus)
fus_df = fus_df[(fus_df['date'] <= '2019-10-11') & (fus_df['date'] >= '2019-10-09')]
fus_df = fus_df.reset_index(drop=True).reset_index()
#display(fus_df[100:150])
#fus_df = fus_df.sort_values('timestamp')
#display(fus_df[100:150])
df1, df2 = process_fus_daily(fus_df)

fus_df[100:150]

test


,index,pid,longitude,latitude,timestamp,date,prev_lat,prev_long,dist,prev_timestamp,delta_timestamp,velocity,stationary
100,100,61762096,17.4781,58.8655,1570777527,2019-10-11,58.8655,17.4781,0.000000,1.570777e+09,0.165278,0.000000,True
101,101,61762096,17.4781,58.8655,1570777832,2019-10-11,58.8655,17.4781,0.000000,1.570778e+09,0.084722,0.000000,True
102,102,61762096,17.4781,58.8655,1570778321,2019-10-11,58.8655,17.4781,0.000000,1.570778e+09,0.135833,0.000000,True
103,103,61762096,17.4781,58.8655,1570778651,2019-10-11,58.8655,17.4781,0.000000,1.570778e+09,0.091667,0.000000,True
104,104,61762096,17.4781,58.8655,1570779032,2019-10-11,58.8655,17.4781,0.000000,1.570779e+09,0.105833,0.000000,True
105,105,61762096,17.4781,58.8655,1570779420,2019-10-11,58.8655,17.4781,0.000000,1.570779e+09,0.107778,0.000000,True
106,106,61762096,17.4781,58.8655,1570779932,2019-10-11,58.8655,17.4781,0.000000,1.570779e+09,0.142222,0.000000,True
107,107,61762096,17.4781,58.8655,1570780232,2019-10-11,58.8655,17.4781,0.000000,1.570780e+09,0.083333,0.000000,True
108,108,61762096,17.4781,58.8655,1570780800,2019-10-11,58.8655,17.4781,0.000000,1.570780e+09,0.157778,0.000000,True
109,109,61762096,17.4781,58.8655,1570781127,2019-10-11,58.8655,17.4781,0.000000,1.570781e+09,0.090833,0.000000,True


In [76]:
%%time

fus_df = format_raw_fus(raw_fus)
#fus_df = fus_df[(fus_df['date'] <= '2019-10-11') & (fus_df['date'] >= '2019-10-09')]
#fus_df = fus_df.sort_values('timestamp')
test = process_fus_daily(fus_df)

test


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

CPU times: user 16.9 s, sys: 0 ns, total: 16.9 s
Wall time: 16.9 s


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in log


## Fused Location Processing

In [118]:
def process_fus_daily(fus, cluster_radius=0.2):
    """Processes daily fused location features.
    
    Assumes date, pid columns are populated.

    This includes average velocity, number of stationary clusters, location entropy, and location variance.

    For reference, see https://github.com/sosata/MobileDepression/tree/master/features and 
    Saeb et al: The relationship between mobile phone location sensor data and depressive symptom severity

    Args:
        fus (pd.DataFrame): the fused location df with latitude, longitude, date, and pid columns

    """
    
    if fus.shape[0]  < 1:
        return None
    
    # get stationary locations
    fus['prev_lat'] = fus['latitude'].shift()
    fus['prev_long'] = fus['longitude'].shift()
    fus['dist'] = fus.apply(lambda x: haversine((x.latitude, x.longitude), (x.prev_lat, x.prev_long)), axis=1) # in km
    fus['prev_timestamp'] = fus['timestamp'].shift()
    fus['delta_timestamp'] = ((fus['timestamp'] - fus['prev_timestamp']) / (60 * 60)).astype(float) # change to hours
    fus['velocity'] = fus['dist'] / fus['delta_timestamp']
    fus = fus[fus['velocity'] >= 0] # drop rows with negative velocities
    fus['stationary'] = fus['velocity'] < 1 # filter locations that have a speed of greater than 1 km/hr
    fus_stationary = fus[fus['stationary']]
    
    #return fus, fus_stationary

    if fus_stationary.shape[0] < 1:
        return None
        
    # TODO should this be stationary or all locations?
    loc_var = np.log(fus_stationary['latitude'].var() + fus_stationary['longitude'].var())

    # assign clusters
    cur_mean = 1
    cur_clusters = 0
    while cur_mean > cluster_radius:
        cur_clusters += 1
        X = fus_stationary[['latitude', 'longitude']]
        kmeans = KMeans(n_clusters=cur_clusters, random_state=0).fit(X)
        labels = kmeans.labels_
        clusters = kmeans.cluster_centers_
        X = X.reset_index(drop=True)
        X['labels'] = labels
        X['center'] = X.apply(lambda x: clusters[int(x.labels)], axis=1)
        X['dist'] = X.apply(lambda x: haversine((x.latitude, x.longitude), x.center), axis=1)
        cur_mean = X['dist'].mean()
    
    # get daily entropy
    fus_stationary = fus_stationary.reset_index(drop=True)
    fus_stationary['cluster'] = X['labels']
    label_group = fus_stationary.groupby(['date', 'cluster'])['delta_timestamp'].sum().unstack()
    label_group = label_group.fillna(0)
    label_group['total'] = label_group.sum(axis=1)
    label_group = label_group.div(label_group['total'], axis=0)
    label_group['entropy'] = -(np.log(label_group) * label_group).sum(axis=1)
    label_group['norm_entropy'] = label_group['entropy'] / np.log(cur_clusters) # entropy normalized by number of location clusters
    label_group = label_group.reset_index()
    
    fus_combined = fus.groupby(['pid', 'date'], as_index=False)['dist'].sum()
    fus_combined = pd.merge(fus_combined, label_group[['date', 'entropy', 'norm_entropy']], on='date', how='outer')
    fus_combined['cluster'] = cur_clusters
    fus_combined['loc_var'] = loc_var
    
    fus_combined['velocity'] = fus.groupby(['pid', 'date'], as_index=False)['velocity'].mean()['velocity']
    #display(fus.groupby(['pid', 'date'], as_index=False)['velocity'].mean())
    #fus_moving = fus[fus['stationary'] > 0]
    
    #fus_combined['transition_time'] = fus_moving.groupby(['pid', 'date'], as_index=False)['delta_timestamp'].sum()
    #display(fus_moving.groupby(['pid', 'date'], as_index=False)['delta_timestamp'].sum())
    
    return fus_combined


def format_raw_fus(fus):
    """Formats raw fus df and returns one ready for processing.

    Args:
        fus (pd.DataFrame): raw fus df pulled from lifesense server

    Returns:
        pd.DataFrame: formatted fus DataFrame
    """    
    fus = format_time(fus)
    
    final_fus = fus
    final_fus['timestamp'] = final_fus['adj_ts']
    
    return final_fus[['pid', 'longitude', 'latitude', 'timestamp', 'date']]


def build_fus_helper(pid, wk, start_date, end_date):
    """Builds fused location features within the given window.
    

    Assumes that the start_date and end_date fall within the given week.
    
    
    Args:
        - pid (str): the participant id
        - wk (int): the study week to pull data from: 1, 4, 7, 10, 13, 16
        - start_date (str): start date, yyyy-mm-dd
        - end_date (str): end date, yyyy-mm-dd
        
    Returns:
        - stats_df (pd.DataFrame): frame with fused location statistics calculated for the given time window
    """
    
    loc = "/data/tliu/wk{}_ls_data/pdk-location/{}.df".format(wk, pid)
    fus_df = pd.read_pickle(loc)
    if fus_df.shape[0] < 1:
        return

    fus_df = format_raw_fus(fus_df)
    fus_df = fus_df[(fus_df['date'] >= start_date) & (fus_df['date'] <= end_date)]
    print(fus_df.shape)
    fus_df["is_wkday"] = (pd.to_datetime(fus_df['date']).dt.dayofweek < 5).astype(float)
    
    wkend_stats = process_fus_daily(fus_df[fus_df["is_wkday"] == 0].copy())
    wkday_stats = process_fus_daily(fus_df[fus_df["is_wkday"] == 1].copy())
    total_stats = process_fus_daily(fus_df.copy())
    
    dfs = []
    
    if wkend_stats is not None:
        wkend_stats = wkend_stats.set_index('pid')
        wkend_stats = wkend_stats.add_suffix("_wkend")
        dfs.append(wkend_stats.mean())
    
    if wkday_stats is not None:
        wkday_stats = wkday_stats.set_index('pid')
        wkday_stats = wkday_stats.add_suffix("_wkday")
        dfs.append(wkday_stats.mean())
    
    if total_stats is not None:
        total_stats = total_stats.set_index('pid')
        total_stats = total_stats.add_suffix("_total")
        dfs.append(total_stats.mean())
    
    stats_df = pd.DataFrame()
    if len(dfs) > 0:
        stats_df = pd.concat(dfs)
        stats_df = stats_df.to_frame().transpose()
        stats_df['pid'] = pid
        stats_df['study_wk'] = wk

    return stats_df
    

In [128]:
%%time

pre_days = 14
post_days = -1
n_procs = 6

fun_args = []
for idx, row in shift_df.iterrows():
    date = row['date']
    pid = row['pid']
    wk = row['study_wk']
    
    start_date = date.floor('D') - pd.Timedelta(pre_days, unit='D')    
    end_date = date.floor('D') + pd.Timedelta(post_days, unit='D')
    
    fun_args.append((pid, wk, start_date, end_date))
   
with multiprocessing.Pool(n_procs) as pool:
    results = pool.starmap(build_fus_helper, fun_args)

df = pd.DataFrame()

for res in results:
    df = df.append(res)



(106, 5)


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(2355, 5)
(1930, 5)
(1874, 5)


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(2940, 5)


...

(3003, 5)


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in log
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: divide by zero encountered in log
/home/tliu/.conda/envs/py37/li

(356, 5)


/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tliu/.conda/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current beha

CPU times: user 8.32 s, sys: 1.42 s, total: 9.74 s
Wall time: 23min 2s


In [129]:
#df.to_pickle("ls_data/shift_fus.df")

In [132]:
df[df['velocity_total'] > 10000].shape

(25, 20)

### Notes

- shift df needs:
    - pid, phq_start, study_wk
- Should standardization of features be per participant or global?
    - likely global
    
- fus correctness analysis:
    - for the problematic frame (negative velocities), there seemed to be an issue with time zone corrections while flying, leading to timestamps "backwards" in time
    - it appears that we can still approximate well the individual's distance traveled and velocity without the negative velocities, so the solution is to drop the negative velocity rows